#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient("mongo")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

### Importer les données

In [5]:
df_ks.dtypes

ID                    int64
name                 object
category             object
main_category        object
currency             object
deadline             object
goal                 object
launched             object
pledged              object
state                object
backers              object
country              object
usd pledged          object
usd_pledged_real    float64
dtype: object

In [6]:
df_ks.iloc[66141,:]

ID                                        85964225
name                Debut Album from Michael Adam 
category                          Grace is Leaving
main_category                           Indie Rock
currency                                     Music
deadline                                       USD
goal                                    2014-04-17
launched                                     700.0
pledged                        2014-04-02 21:56:35
state                                        850.0
backers                                 successful
country                                         32
usd pledged                                     US
usd_pledged_real                               850
Name: 66141, dtype: object

In [7]:
df_ks.drop(66141,0,inplace=True)

In [8]:
df_ks.iloc[66141,:]

ID                                                         1863139290
name                O2 The Dance Company presents: Between the Bra...
category                                                        Dance
main_category                                                   Dance
currency                                                          CAD
deadline                                                   2015-04-01
goal                                                          15000.0
launched                                          2015-02-27 04:06:14
pledged                                                       15080.0
state                                                      successful
backers                                                            34
country                                                            CA
usd pledged                                             12076.7337028
usd_pledged_real                                              11926.6
Name: 66142, dtype: 

In [9]:
df_ks2 = df_ks.astype({"usd pledged":"float64"})
df_ks2 = df_ks2.astype({"pledged":"float64"})
df_ks2 = df_ks2.astype({"goal":"float64"})
df_ks2 = df_ks2.astype({"backers":"int64"})

In [10]:
df_ks2.drop(["currency"],axis='columns',inplace=True)

In [11]:
df_ks2.dtypes

ID                    int64
name                 object
category             object
main_category        object
deadline             object
goal                float64
launched             object
pledged             float64
state                object
backers               int64
country              object
usd pledged         float64
usd_pledged_real    float64
dtype: object

In [12]:
dict_df = df_ks2.to_dict('record')

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/pandas/core/frame.py:1485: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  warnings.warn(


In [14]:
dict_df

[{'ID': 872782264,
  'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
  'category': 'Rock',
  'main_category': 'Music',
  'deadline': '2011-09-16',
  'goal': 2000.0,
  'launched': '2011-08-17 06:31:31',
  'pledged': 1145.0,
  'state': 'canceled',
  'backers': 24,
  'country': 'US',
  'usd pledged': 1145.0,
  'usd_pledged_real': 1145.0},
 {'ID': 1326492673,
  'name': 'Ohceola jewelry',
  'category': 'Fashion',
  'main_category': 'Fashion',
  'deadline': '2012-08-22',
  'goal': 18000.0,
  'launched': '2012-07-23 20:46:48',
  'pledged': 1851.0,
  'state': 'failed',
  'backers': 28,
  'country': 'US',
  'usd pledged': 1851.0,
  'usd_pledged_real': 1851.0},
 {'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': '2016-07-01 21:55:54',
  'pledged': 7534.0,
  'state': 'successful',
  'backers': 254,
  'country': 'US',
  'usd p

In [15]:
collection.insert_many(dict_df)

In [16]:
collection.find_one()

{'_id': ObjectId('5fcb976bc82262f665e27601'),
 'ID': 872782264,
 'name': 'Scott Cooper\'s Solo CD "A Leg Trick" (Canceled)',
 'category': 'Rock',
 'main_category': 'Music',
 'deadline': '2011-09-16',
 'goal': 2000.0,
 'launched': '2011-08-17 06:31:31',
 'pledged': 1145.0,
 'state': 'canceled',
 'backers': 24,
 'country': 'US',
 'usd pledged': 1145.0,
 'usd_pledged_real': 1145.0}

## Question 1  

In [54]:
max = collection.find({},{"name","pledged"}).sort("pledged",-1).limit(5)
for i in max :
    print("--------")
    print(i)

--------
{'_id': ObjectId('5fcb976bc82262f665e3a125'), 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'pledged': 13285226.36}
--------
{'_id': ObjectId('5fcb976cc82262f665e3fc1e'), 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'pledged': 12779843.49}
--------
{'_id': ObjectId('5fcb976bc82262f665e3ad5c'), 'name': 'Expect the Unexpected. digiFilmï¿½ Camera by YASHICA', 'pledged': 10035296.0}
--------
{'_id': ObjectId('5fcb976cc82262f665e43139'), 'name': 'OUYA: A New Kind of Video Game Console', 'pledged': 8596474.58}
--------
{'_id': ObjectId('5fcb976cc82262f665e4a8f0'), 'name': 'The Everyday Backpack, Tote, and Sling', 'pledged': 6565782.5}


## Question 2

In [55]:
cur = collection.find({"state":"successful"})
list(cur)

[{'_id': ObjectId('5fcb976bc82262f665e27603'),
  'ID': 1688410639,
  'name': 'Sluff Off & Harald: Two latest EGGs are Classics "old & new"',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'deadline': '2016-07-19',
  'goal': 2000.0,
  'launched': '2016-07-01 21:55:54',
  'pledged': 7534.0,
  'state': 'successful',
  'backers': 254,
  'country': 'US',
  'usd pledged': 3796.0,
  'usd_pledged_real': 7534.0},
 {'_id': ObjectId('5fcb976bc82262f665e27604'),
  'ID': 156812982,
  'name': 'SketchPlanner: Create and Plan- all in one beautiful book!',
  'category': 'Art Books',
  'main_category': 'Publishing',
  'deadline': '2017-09-27',
  'goal': 13000.0,
  'launched': '2017-08-28 15:47:02',
  'pledged': 16298.0,
  'state': 'successful',
  'backers': 367,
  'country': 'US',
  'usd pledged': 2670.0,
  'usd_pledged_real': 16298.0},
 {'_id': ObjectId('5fcb976bc82262f665e27609'),
  'ID': 1669374237,
  'name': 'Help I Am Wolves fund their first album "ABCD" on 12" vinyl!',
  'category':

In [64]:
collection.find({"state":"successful"}).count()

<ipython-input-64-fdf8cb1550e9>:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  collection.find({"state":"successful"}).count()


53040

## Question 3

In [74]:
CountCategory = collection.aggregate([{"$group":{"_id":"$category","count":{"$sum":1}}}])

In [75]:
list(CountCategory)

[{'_id': 'Nature', 'count': 219},
 {'_id': 'Photography', 'count': 2239},
 {'_id': 'Musical', 'count': 367},
 {'_id': 'Fantasy', 'count': 132},
 {'_id': 'Pop', 'count': 1302},
 {'_id': 'Graphic Novels', 'count': 702},
 {'_id': 'Drama', 'count': 871},
 {'_id': 'Art', 'count': 3358},
 {'_id': 'Mobile Games', 'count': 650},
 {'_id': 'Flight', 'count': 158},
 {'_id': 'Drinks', 'count': 995},
 {'_id': 'Classical Music', 'count': 1064},
 {'_id': 'Apps', 'count': 2535},
 {'_id': 'Theater', 'count': 2786},
 {'_id': 'Accessories', 'count': 1204},
 {'_id': 'Hardware', 'count': 1431},
 {'_id': 'Fashion', 'count': 3379},
 {'_id': 'Product Design', 'count': 8886},
 {'_id': 'Faith', 'count': 439},
 {'_id': 'Family', 'count': 130},
 {'_id': 'Typography', 'count': 44},
 {'_id': 'DIY', 'count': 480},
 {'_id': "Children's Books", 'count': 2686},
 {'_id': 'Technology', 'count': 2690},
 {'_id': 'Live Games', 'count': 394},
 {'_id': 'Space Exploration', 'count': 137},
 {'_id': 'Sculpture', 'count': 737},
 

## Question 4

In [97]:
FrProject = collection.find({"country":"FR"})
list(FrProject)

[{'_id': ObjectId('5fcb976bc82262f665e276d9'),
  'ID': 1131677007,
  'name': 'Bittersweet',
  'category': 'Shorts',
  'main_category': 'Film & Video',
  'deadline': '2016-02-04',
  'goal': 2000.0,
  'launched': '2016-01-05 06:00:18',
  'pledged': 2293.0,
  'state': 'successful',
  'backers': 44,
  'country': 'FR',
  'usd pledged': 2492.60262324,
  'usd_pledged_real': 2568.6120757253275},
 {'_id': ObjectId('5fcb976bc82262f665e2783d'),
  'ID': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'deadline': '2015-07-01',
  'goal': 20000.0,
  'launched': '2015-05-27 11:29:28',
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': ObjectId('5fcb976bc82262f665e27877'),
  'ID': 1442243866,
  'name': "GOB'Z'HEROES - The Game",
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'deadline': '2016-01-31',
  'goal': 30000.0,
  'launche

In [95]:
CountProject = collection.find({"$and":[{"launched":{"$lt":"2016-01-01"}},{"country":"FR"}]})

In [96]:
list(CountProject)

[{'_id': ObjectId('5fcb976bc82262f665e2783d'),
  'ID': 545482362,
  'name': 'Depeche Mode Bar Belgium',
  'category': 'Pop',
  'main_category': 'Music',
  'deadline': '2015-07-01',
  'goal': 20000.0,
  'launched': '2015-05-27 11:29:28',
  'pledged': 305.0,
  'state': 'failed',
  'backers': 10,
  'country': 'FR',
  'usd pledged': 333.98121895,
  'usd_pledged_real': 338.42637284599937},
 {'_id': ObjectId('5fcb976bc82262f665e279e7'),
  'ID': 400288569,
  'name': 'Dracucat : A vampire kitten card game',
  'category': 'Tabletop Games',
  'main_category': 'Games',
  'deadline': '2015-10-29',
  'goal': 1900.0,
  'launched': '2015-09-29 07:04:08',
  'pledged': 2944.0,
  'state': 'successful',
  'backers': 129,
  'country': 'FR',
  'usd pledged': 3295.37237632,
  'usd_pledged_real': 3243.3980764357875},
 {'_id': ObjectId('5fcb976bc82262f665e27ba7'),
  'ID': 1990332786,
  'name': 'Minotaur enjoy your life & your ice cream',
  'category': 'Food Trucks',
  'main_category': 'Food',
  'deadline': '2

## Question 5

In [83]:
UsProject = collection.find({"$and":[{"goal":{"$gt":200000}},{"country":"US"}]})

In [84]:
list(UsProject)

[{'_id': ObjectId('5fcb976bc82262f665e276a3'),
  'ID': 866634482,
  'name': 'A CALL TO ADVENTURE',
  'category': 'Film & Video',
  'main_category': 'Film & Video',
  'deadline': '2012-09-14',
  'goal': 287000.0,
  'launched': '2012-08-13 23:14:02',
  'pledged': 1465.0,
  'state': 'failed',
  'backers': 11,
  'country': 'US',
  'usd pledged': 1465.0,
  'usd_pledged_real': 1465.0},
 {'_id': ObjectId('5fcb976bc82262f665e27756'),
  'ID': 993194166,
  'name': 'Storybricks, the storytelling online RPG',
  'category': 'Video Games',
  'main_category': 'Games',
  'deadline': '2012-06-01',
  'goal': 250000.0,
  'launched': '2012-05-01 20:49:58',
  'pledged': 23680.54,
  'state': 'failed',
  'backers': 409,
  'country': 'US',
  'usd pledged': 23680.54,
  'usd_pledged_real': 23680.54},
 {'_id': ObjectId('5fcb976bc82262f665e2775e'),
  'ID': 1147175344,
  'name': 'Shine On New World',
  'category': 'Theater',
  'main_category': 'Theater',
  'deadline': '2013-11-04',
  'goal': 300000.0,
  'launched'

## Question 6 

In [78]:
collection.create_index([("name", "text")])

'name_text'

In [79]:
collection.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]},
 'name_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'weights': SON([('name', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [81]:
sport = collection.find( { "$text": { "$search": "sport" } } )
list(sport)

[{'_id': ObjectId('5fcb976bc82262f665e32556'),
  'ID': 1875366029,
  'name': 'Sport Smart. A New Genre of Sports TV. Sport Fans Unite!!!',
  'category': 'Webseries',
  'main_category': 'Film & Video',
  'deadline': '2011-01-15',
  'goal': 4500.0,
  'launched': '2010-12-16 08:40:02',
  'pledged': 25.0,
  'state': 'failed',
  'backers': 1,
  'country': 'US',
  'usd pledged': 25.0,
  'usd_pledged_real': 25.0},
 {'_id': ObjectId('5fcb976bc82262f665e2a738'),
  'ID': 1126822169,
  'name': 'Frey Sports App - We connect sports people.',
  'category': 'Apps',
  'main_category': 'Technology',
  'deadline': '2017-01-11',
  'goal': 25000.0,
  'launched': '2016-12-12 11:30:28',
  'pledged': 0.0,
  'state': 'failed',
  'backers': 0,
  'country': 'DK',
  'usd pledged': 0.0,
  'usd_pledged_real': 0.0},
 {'_id': ObjectId('5fcb976cc82262f665e48db6'),
  'ID': 1081541783,
  'name': 'Daily Fantasy Sports | Sports Analytics Platform | DFS',
  'category': 'Web',
  'main_category': 'Technology',
  'deadline':